#Importing necessary libraries and modules

In [ ]:
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
 
 
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression


import nltk
nltk.download('punkt')
nltk.download('stopwords')

!python -m spacy download en_core_web_sm
import spacy 
from spacy import displacy


!pip install goslate
!pip install googletrans==3.1.0a0
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning) 


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
  Created wheel for goslate: filename=goslate-1.5.1-cp37-none-any.whl size=11412 sha256=be9e78486a581dd899e0beed2b2a3f0ef48a68d012a69185d2d8a1e7e72e008e
  Stored in directory: /root/.cache/pip/wheels/4f/7f/28/6f52271012a7649b54b1a7adaae329b4246bbbf9d1e4f6e51a
Successfully built goslate
     |████████████████████████████████| 61kB 3.0MB/s 
     |████████████████████████████████| 1.0MB 8.5MB/s 
     |████████████████████████████████| 51kB 5.7MB/s 
     |████████████████████████████████| 71kB 5.5MB/s 
     |████████████████████████████████| 61kB 6.7MB/s 
  Created wheel for googletrans: filename=googletrans-3.1.0a0-cp37-none-any.whl size=16368 sha256=62920c63e70a4d761363abfeea8d

#Translating text from one language to another

FUNCTION TO TRANSLATE A SINGLE QUESTION(STRING)

In [ ]:
def translate_2(to_translate, to_langage="auto", langage="auto"):
        import goslate
        gs = goslate.Goslate()
     #   print(gs.translate(to_translate, 'en'))
        result = gs.translate(to_translate,'en')
        return result
translate_2("భారత దేశ ప్రాధాని ఎవరు?")


'Who is the Prime of India?'

FUNCTION TO TRANSLATE TWO LISTS FROM TELUGU TO ENGLISH

In [ ]:
def translate(quest_list,ans_list):
  from googletrans import Translator
  translater=Translator()
  out_quest=[] #tranlated list
  out_ans=[]
  for quest in quest_list:
    translated=translater.translate(quest,dest="en")
    out_quest.append(translated.text)
  for ans in ans_list:
    translated=translater.translate(ans,dest="en")
    out_ans.append(translated.text)
  return out_quest,out_ans
(Target_Questions,Target_Answers)=translate(["భారత దేశ ప్రధాని ఎవరు?","అడ్వెంచర్స్ ఆఫ్ టామ్ సాయర్” రచయిత ఎవరు?"],["మోడీ"])
print(Target_Questions,"and",Target_Answers)

['Who is the Prime Minister of India?', "Who is the author of 'Adventures of Tom Sawyer'?"] and ['Modi']


FUNCTION TO TRANSLATE A STRING FROM ENGLISH TO TELUGU LANGUAGE

In [ ]:
def translate2(quest):
  from googletrans import Translator
  translater=Translator()
  out=translater.translate(quest,dest="te")
  print(out.text)
  return out.text
print(translate2("country"))


దేశం
దేశం


**#FUNCTION TO CLASSIFY THE ANSWER TYPE OF THE QUESTION**

In [ ]:
def Question_Classification(user_input):
  qa=open("/content/drive/MyDrive/Odqa_telugu_Train_set.txt","a")
  qa.write("    :"+user_input+"\n")
  qa.close()
  f=open("/content/drive/MyDrive/Odqa_telugu_Train_set.txt","r")
  classes=[]
  queries=[]
  new_label=[]
  for line in f:

    line=line.rstrip('\n')
    classes.append((line.split()[0]).split(":")[0])
    lb=(line.split()[0]).split(":")[0]
    #print(lb)
    if(lb=="PERS"):
      new_label.append(1)
    if(lb=="LOCA"):
      new_label.append(2)
    if(lb=="ORGA"):
      new_label.append(3)
    if(lb=="DATE"):
      new_label.append(4)
    if(lb=="TIME"):
      new_label.append(5)
    if(lb=="PERC"):
      new_label.append(6)
    if(lb=="NUMB"):
      new_label.append(7)
    if(lb=="CURR"):
      new_label.append(8)
    queries.append(line[5:])

  vectorizer =TfidfVectorizer(min_df=1)# min_df is 1, which means "ignore terms that appear in less than 1 document". Thus, the default setting does not ignore any terms.
  
  X = vectorizer.fit_transform(classes) #Hence text converted to vectors to process
  Y = vectorizer.fit_transform(queries)

  X_ = Y[:len(new_label)]
  Y_ = new_label[:len(new_label)]
  X_train, X_test, y_train, y_test = train_test_split(X_, Y_, test_size=0.25, random_state=42)
#	print("X_train = ",X_train.shape)
#	print("X_test = ",X_test.shape)

  test=Y[len(queries)-1].toarray()


  #nn=MLPClassifier(activation='tanh',solver='sgd',hidden_layer_sizes=(80,50,30,8),random_state=4,alpha=0.1,batch_size=71)#73.9 accuracy
	
  nn = LinearSVC(random_state=0, tol=0.3,loss="squared_hinge",multi_class="crammer_singer")#74.4 73.5 76.1
  #nn = LogisticRegression(random_state=0, solver='sag', multi_class='multinomial')#71.4 73.9
  

	

  nn.fit(X_train,y_train)
  pred2=nn.predict(X_test)


  hits=0.00
  for i in range(0,len(y_test)):
    if y_test[i]==pred2[i]:
      hits=hits+1
			

  #print("The accuracy is ",((hits/len(y_test))*100.0))
  
  
	
  result=nn.predict(test)
  if(result==1):
    ans_type.append("PERSON")

  elif(result==2):
    ans_type.append("LOCATION")

  elif(result==3):
    ans_type.append("ORGANIZATION")

  elif(result==4):
    ans_type.append("DATE")

  elif(result==5):
    ans_type.append("TIME")

  elif(result==6):
    ans_type.append("PERCENTAGE")
  
  elif(result==7):
    ans_type.append("NUMBER")
  else:
    ans_type.append("CURRENCY")
  return(ans_type[0])

  
user_input_telugu="“అడ్వెంచర్స్ ఆఫ్ టామ్ సాయర్” రచయిత ఎవరు?" #input("enter query")
ans_type=[]
Question_Classification(user_input_telugu)


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


'PERSON'

#FUNCTION TO FIND SEQUENCE MATCH SCORE BETWEEN TWO STRINGS

In [ ]:

#TO find Sequence matching scores
def similarity(str1,str2):
  str1=str1.lower()
  str2=str2.lower()
  #print(str1,str2)
  
  from difflib import SequenceMatcher
  score=SequenceMatcher(lambda x: x==" ",str1,str2).ratio()
  
  return score
similarity("venu","M.Venu")



0.8

# FUCTION TO FIND NER TAGS to the WORDS IN A STRING


In [ ]:
def ner_tag(sents,tag):
  import spacy 
  from spacy import displacy
  if(tag=="LOCATION"):
    tag="GPE"
  if(tag=="NUMBER"):
    tag="CARDINAL"
  if(tag=="ORGANIZATION"):
    tag="ORG"
  if(tag=="CURRENCY"):
    tag="MONEY"

  

  # Load SpaCy model
  nlp = spacy.load("en_core_web_sm")
  #nlp = spacy.load("en_core_web_md")
  #nlp = spacy.load("en_core_web_lg")
  import pandas as pd
  
  doc = nlp(sents)

  entities = []
  labels = []
  position_start = []
  position_end = []

  for ent in doc.ents:
    entities.append(ent)
    labels.append(ent.label_)
    position_start.append(ent.start_char)
    position_end.append(ent.end_char)
    #print(entities, labels)
  
  ans=[]  
  df = pd.DataFrame({'Entities':entities,'Labels':labels}) #,'Position_Start':position_start, 'Position_End':position_end})
  for i in range(0,len(labels)):
    if labels[i]==tag:
      ans.append(entities[i])
  #print(df)

  

    
  print("possible answers list is",ans)
  #print("final answer is",ans[0])
  #final_ans=ans[0]
      



 
  return ans #LIST OF ANSWERS
#ner_tag("narendra modi on 2-03-2021  gave 1 billion dollars, RS.900, $351 AND 500 dollars supreme court is prime minister  Rs 540 crore of Lom Turkish Togo  lira of india","ORGANIZATION") 
#ner_tag("Lome 1650","LOCATION")
ner_tag("\n Mark  Twain, MARK TWAIN AND  Narendra modi money 500 dollars should NOT exceed INR 4000 USD 100 per annum.","PERSON")

                Entities   Labels
0             (Narendra)      ORG
1  (money, 500, dollars)    MONEY
2                  (INR)      ORG
3       (4000, USD, 100)  PRODUCT
possible answers list is []


[]

In [ ]:
def curr_ner(words_list):
  #print(words_list)
  ans_list=[]
  
  train=['Turkish lira','Turkish','lira','Birr', 'peso', 'Metical', 'Mark', 'Balboa', 'Ruble', 'Euro', 'Krona', 'Lake', 'Colon', 'Euro', 'Sudanese pound', 'Kina', 'Corona', 'Turkish lira', 'Deutsche Mark', 'Malaysian Dollar', 'Kuwaiti Dinar', 'Inti Soul', 'Drachma', 'pound', 'Bolivar', 'Real', 'New Shekel', 'peso', 'Colon', 'pound', 'Tugrick', 'French Frank', 'American Dollar', 'Shilling', 'Rs. 1000', 'American Dollar', 'Rs 2 crore', 'Rs 540 crore', 'Rs. 115', 'Rs.5,000','₹','₹1234567890','750-million-dollar', '50 pence', 'Rs.650 crore', '1 billion', 'USD 75 million', 'Rs.50,000 crores', 'Rs 2 trillion', 'Rs 2 trillion', 'Euro', '1,01,500', '1,8292 crores', 'USD 15 million']
  #print(train)
  i=0
  for word in words_list:
    for word_in_train in train:
      sc=similarity(word_in_train,word)
      #print(word_in_train,word,sc)
      if(sc>=0.7):
        ans_list.append(word)
    i+=1
  print("Possible answers list is ",ans_list)
  return (ans_list)
curr_ner(['Birr',"nothing"])

Possible answers list is  ['Birr']


['Birr']

**#MAIN CODE FOR PARAGRAPH EXTRACTION USING SEARCH ENGINES 1.BING   2.GOOGLE**

In [ ]:
#Main code starts here
import requests, webbrowser
import nltk
import urllib
#nltk.download('punkt')
from bs4 import BeautifulSoup
import random
def getting_links(search):
  
  A = ("Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
       "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
       "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
      )

  Agent = A[random.randrange(len(A))]
  headers={'user-agent': Agent}
  
  r = requests.get(url,headers=headers)
  soup = BeautifulSoup(r.content, 'lxml')
  
  if not(soup.findAll('li', attrs={'class':'b_algo'})):getting_links(search)
	#print("Not able to find urls, Some Error came call back the function again....\n")
  else: 
    for listt in soup.findAll('li', attrs={'class':'b_algo'}):
      head_links = listt.find('h2')
      for link in head_links.find_all('a', href=True):
        final_links.append(link['href'])
  
  Numlinks=len(final_links[:-2])
  return final_links[:no_of_url]





#A = ("Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2228.0 Safari/537.36",
#      "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.1 Safari/537.36",
 #      "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/41.0.2227.0 Safari/537.36",
  #    )

#Agent = A[random.randrange(len(A))]
#headers={'user-agent': Agent}

#downloaded = drive.CreateFile({'id':'1HBH06-0wdBrC-S4oJNX3pDTdSWry5f5d'}) # replace the id with id of file you want to access
#downloaded.GetContentFile('OVER_ALL_QA_DATA.txt') 
#t = open("OVER_ALL_QA_DATA.txt","r").readlines()
#Target_Questions_telugu = []
#Target_Answers_telugu	 = []
#for line in t:
#	line = line.strip()
#	line = line.split(":::")
#	Target_Answers_telugu.append(line[0].strip())
#	Target_Questions_telugu.append(line[1].strip())


 
t = open("/content/drive/MyDrive/odqa_telugu_testset.txt","r").readlines()
print(len(t))
Target_Questions_telugu = []
Target_Answers_telugu	 = []

for line in t:
  line=line.strip()
  line=line.split("@@@")
  Target_Questions_telugu.append(line[0].strip()[5:])
  Target_Answers_telugu.append(line[1].strip())

t2 = open("/content/drive/MyDrive/odqa_telugu_testset.te.en.txt","r").readlines()
print(len(t2))
Target_Questions = []
Target_Answers	 = []

for line in t2:
  line=line.strip()
  line=line.split("?")
  Target_Questions.append(line[0].strip()[5:]+"?")
  Target_Answers.append(line[1].strip())


#t = open("/content/drive/MyDrive/odqa_telugu_testset_english.txt","r").readlines()

n=50 #input("no of questions to be checked")
hit=0
partial=0
partial_2=0
engine=input("select search engine google/bing ")
help=int(input("ENTER starting position of tag(like 250 for curr,200 for car,150 for date,100 for org,50 for loc,0 for pers"))
help2="" #USEFUL WHILE FINDING QUESTION CLASSIFICATION ACCURACY
#(Target_Questions,Target_Answers)=translate(Target_Questions_telugu[0:n],Target_Answers_telugu[0:n]) # Test questions and Test answers are converted from telugu to english
#print("checking for conversion to english from telugu")
print(Target_Questions[0+help:n+help])
#print(Target_Answers[0:n])

z=0
count=0
final_answer_list=[]
for z in range(0,n):
  if(z>=0 and z<50):#These conditions useful to find Quesion classification accuracy
    help2="PERSON"
  if(z>=50 and z<100):
    help2="LOCATION"
  if(z>=100 and z<150):
    help2="ORGANIZATION"
  if(z>=150 and z<200):
    help2="DATE"
  if(z>=200 and z<250):
    help2="NUMBER"
  if(z>=250 and z<300):
   help2="CURRENCY"  
  z=z+help

  print("\nsearching for question",z+1,"..............,")
  print("Question: ",Target_Questions[z])

  
  user_input_telugu=Target_Questions_telugu[z]
  user_input=Target_Questions[z]    #input("Enter your Query in telugu ")
  
  ans_type=[]
  tag=Question_Classification(user_input_telugu)
  print(tag)
  
  if(tag==help2):
    count+=1
  #print("Quesiotn classification accuracu is ",count*100/50)
  
  #user_input=translate_2(user_input_telugu)
  
  if(engine=='google'):
    google_search=requests.get("https://www.google.com/search?q="+user_input)
    # print(google_search.text)
    soup=BeautifulSoup(google_search.content,'html.parser')
    no_of_url=3 #int(input("Enter the no of url to get information "))
    no_of_sentences=15 #int(input("enter no of sentences to be displayed "))
    templinks=[]
    for link in soup.find_all('a'):
      if 'href' in link.attrs:
        templinks.append(link.attrs['href'])
    #print(templinks)
    links=[]
    for link in templinks:
      if (link[1]=='u'): 
          links.append(link)
    #print(links)
    final_links=[];
    for link in links:
      x=link.find('&')
      if(x!=-1):
        tempstring=link[7:x]
        if tempstring not in final_links and (tempstring.find('%')==-1):
          if len(final_links)<no_of_url:
            final_links.append(link[7:x])
          else:
            break
    #print(final_links)
    
    
  
  elif engine=='bing':
    no_of_url=3 #int(input("Enter the no of url to get information "))
    no_of_sentences=15 #int(input("enter no of sentences to be displayed "))
    prelink=f"https://www.{engine}.com/search?"
    query_args = { 'q':user_input, 'num': no_of_url }
    encoded_args = urllib.parse.urlencode(query_args)
    url = prelink + encoded_args
    final_links=[]
    getting_links(user_input)
    final_links=final_links[0:no_of_url]
    #print("lenght of final links list is",len(final_links))
    #print(final_links)
    

    
  
  
  else:
    print("you have selected wrong search engine")
    exit()

  #print("final links are ",final_links)
  #soup=[]
  #for link in final_links:
  #  try:
  #   webopen=requests.get(link)
    #  soup.append(BeautifulSoup(webopen.content,'lxml'))
    #except:
    #  pass
  soup=[]
  for link in final_links:
    try:
        webopen=requests.get(link)
        soup.append(BeautifulSoup(webopen.content,'lxml'))
    except:
        pass


  similar={} 
  titles=[]
  for i in range(len(soup)):#looping around each website
    try:
    #text=soup[i].get_text()
      text=[]
      #print(soup[i].title.text)
      titles.append(soup[i].title.text[0:-11])
      for para in soup[i].findAll('p'):# paragraph extration
          pa=para.text
          text.append(pa) #text is a list whose elements are paragraphs(which contains sentences)
      
      from nltk import sent_tokenize
      from nltk import word_tokenize
      sentence=[]
      for para in text:
        sentence=sent_tokenize(para)# divided content of website into a sentences
        #print(len(sentence))
        #print(sentence)
      
      # nltk.download('stopwords')
        from nltk.corpus import stopwords
        user_input_words=word_tokenize(user_input)
        sw=stopwords.words('english')
        user_input_final_words={w.lower() for w in user_input_words if not w in sw } #sets
      
        sentence_words=[]
        sentence_final_words=[] #list in which stop words are removed
        i=0
        
        for i in range(0,len(sentence)):
          sentence_words.append(word_tokenize(sentence[i]))
          sentence_final_words.append({w.lower() for w in sentence_words[i] if not w in sw})
          rvector = user_input_final_words.union(sentence_final_words[i]) 
          v1=[]
          v2=[]
          for w in rvector:
            if w in user_input_final_words:
              v1.append(1)
            else:
              v1.append(0)
            if w in sentence_final_words[i]:
              v2.append(1)
            else:
              v2.append(0)
          c=0
          for j in range(len(rvector)):
            c+=v1[j]*v2[j]
          v1_square=[v**2 for v in v1]
          v2_square=[v**2 for v in v2]
          try:
            cosine=c/(float((sum(v1_square)*sum(v2_square))**0.5))
            similar[sentence[i]]=cosine
          except:
            pass
    except:
      pass

    #print(cosine) 
    #print(len(cosine))
    #print(max(cosine))
  
    #print(sentence[cosine.index(max(cosine))])
  #print(max(similar.values()))

  sorted_dict = dict( sorted(similar.items(),
                            key=lambda item: item[1],
                            reverse=True))
  relevant_sent=list(sorted_dict.keys())[:no_of_sentences]
  text_of_sent=""
  for sent in relevant_sent:
    #print(sent)
    text_of_sent=text_of_sent+" "+sent
  for title in titles:
    text_of_sent=text_of_sent+" "+title

  #final_ans=ner_tag(text_of_sent,tag)
  
  try:
    if(tag=='CURRENCY'):
      #print("OKKKKK")
      words_list=word_tokenize(text_of_sent)
      #temp=str(ner_tag(text_of_sent,tag)) #Answer produced by spacy ner for currencu tag
      #words_list.append(temp)
      ans=curr_ner(words_list)
    else:
      ans=ner_tag(text_of_sent,tag)
    #print(df)
    
    
#    if(str(Target_Answers[z])==str(ans)):      #str(Target_Answers[z]).find(str(ans))!=-1 or str(ans).find(Target_Answers[z])!=-1 ):
#      hit+=1
  except:
    ans=["NO ANSWER"]
    #final_answer_list.append(ans)
    pass
  try:
    print('final answer is', ans[0])
  except:
    ans.append("NO ANSWER")
    print('final answer is ',ans[0])
  print("Actual answer is",Target_Answers[z])
  final_answer_list.append(str(ans[0]))

  similar={}
  target_ans_words=set()
  final_ans_words=set()
  target_ans_words=set(word_tokenize(Target_Answers[z]))
  final_ans_words=set(word_tokenize(str(ans[0])))
  rvector=set()
  rvector = target_ans_words.union(final_ans_words) 
  v1=[] #for target answers
  v2=[]
  
  for w in rvector:
    if w in target_ans_words:
      v1.append(1)
    else:
      v1.append(0)
    if w in final_ans_words:
      v2.append(1)
    else:
      v2.append(0)
  c=0
  j=0
  try:
    for j in range(len(rvector)):
      c+=v1[j]*v2[j]
    v1_square=[v**2 for v in v1]
    v2_square=[v**2 for v in v2]
      
    cosine2=c/(float((sum(v1_square)*sum(v2_square))**0.5))
    print("cosine similarity is ",cosine2)
     # similar[z]=cosine2
  except:
    pass
  score=similarity(Target_Answers[z],str(ans[0]))
  print("Sequence matcher score is ",score)
  if cosine2==1.0 or score==1.0:
    hit+=1
  if cosine2>=0.7 or score>=0.7:
    partial+=1
  if cosine2>=0.4 or score>=0.5:
    partial_2+=1
  print("hits are ",hit,partial,partial_2)
        


print("hits are",hit)
accuracy= hit*100/n
print("actual answers list is,",Target_Answers[0+help:n+help])
print("answers that I got are",final_answer_list)
print("Exact match accuracy for",n,"no of questions is ",accuracy)

print("no of partial correct ans are ",partial)
partial_accuracy=partial*100/n
print("partial match accurccy is for ",n,"no of questions is ",partial_accuracy)

print("no of partial_2(cosine>=0.5) are ",partial_2)
partial_2_accuracy=partial_2*100/n
print("partial_2 accuracy is for ",n,"no of questioons is ",partial_2_accuracy)
print("Quesiton classification accuracy is ",(count*100)/50)









300
300
select search engine google/bing bing
ENTER starting position of tag(like 250 for curr,200 for car,150 for date,100 for org,50 for loc,0 for pers250
[' Turkish currency?', ' Which is the Ethiopian currency?', ' What is the currency of Chile?', ' What is the currency of Mozambique?', ' What is the currency of Finland?', ' What is the currency of Panama?', ' What is the currency of Russia?', ' Which is the official currency of the European Union?', ' What is the Swedish currency?', ' What is the currency of Albania?', ' Which is the El Salvador currency?', ' What is the Dutch currency?', ' What is the Sudanese currency?', ' What is the currency of Papua New Guinea?', ' What is the currency of Czechoslovakia?', ' What is the currency of Turkey?', ' What is the German currency (United)?', ' What is the currency of Malaysia?', ' What is the Kuwaiti currency?', ' What is the currency of Peru?', ' What is the Greek currency?', ' What is the currency of Lebanon?', ' What is the currenc

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


OKKKKK
Possible answers list is  ['Turkish', 'Turkish', 'Turkish', 'lira', 'Turkish', 'Turkish', 'Turkish', 'Lira', 'Turkish', 'Turkish', 'Turkish', 'lira', 'Turkish', 'Turkish', 'Turkish', 'lira', '200-Turkish-lira', '200-Turkish-lira', 'Turkish', 'Turkish', 'Turkish', 'lira', 'Turkish', 'Turkish', 'Turkish', 'Turkish', 'Turkish', 'Turkish', 'lira', 'removal', 'Turkish', 'Turkish', 'Turkish', 'lira', 'Turkish', 'Turkish', 'Turkish', 'Lira', 'Turkish', 'Turkish', 'Turkish', 'Turkish', 'Turkish', 'Turkish', 'lira', 'lirası', 'Turkish', 'Turkish', 'Turkish', 'lira', 'Turkish', 'Turkish', 'Turkish', 'lira', 'lira', 'pound', 'pound', 'Turkish', 'Turkish', 'Turkish', 'lira', 'Turkish', 'Turkish', 'Turkish', 'lira', 'lira', 'Turkish', 'Turkish', 'Turkish', 'around', 'around', 'Turkish', 'Turkish', 'Turkish', 'Lira', 'Turkish', 'Turkish', 'Turkish', 'lira', 'make', 'make', 'Turkish', 'Turkish', 'Turkish', 'Lira', 'Turkish', 'Turkish', 'Turkish', 'lira', 'Turkish', 'Turkish', 'Turkish', 'liras

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


OKKKKK
Possible answers list is  ['Birr', 'Birr', 'Birr', 'Birr', 'Birr', 'birr', 'birr', 'Birrs', 'Birr', 'Birr', 'birr', 'Birr']
final answer is Birr
Actual answer is Birr
cosine similarity is  1.0
Sequence matcher score is  1.0
hits are  1 2 2

searching for question 253 ..............,
Question:   What is the currency of Chile?
CURRENCY


...

                                             Entities    Labels
0   (Mahatma, Gandhi, National, Rural, Employment,...       ORG
1                                       (2019, -, 20)      DATE
2                                            (61,500)  CARDINAL
3                                           (MGNREGA)       ORG
4                                    (last, year, 's)      DATE
5                                            (71,002)  CARDINAL
6                                   (13.4, per, cent)     MONEY
7                                            (60,000)  CARDINAL
8                   (the, Union, Budget, 2019, -, 20)       ORG
9                                            (11,002)  CARDINAL
10                                        (the, year)      DATE
11                                           (267.98)  CARDINAL
12                   (the, fiscal, year, 2018, -, 19)      DATE
13                                     (Person, days)      DATE
14                                      

/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


                           Entities       Labels
0                           (India)          GPE
1                             (JJM)          ORG
2                     (Maharashtra)          ORG
3                     (2020, -, 21)         DATE
4                           (India)          GPE
..                              ...          ...
60  (Meghalaya, ,, Assam, &, Bihar)          ORG
61           (Jal, Jeevan, Mission)       PERSON
62                   (Jharkhand, &)          ORG
63            (GK, Questions, 2020)  WORK_OF_ART
64                       (June, 18)         DATE

[65 rows x 2 columns]
possible answers list is [JJM, Maharashtra, JJM, State, JJM, Government of India, India News Centre, AAP, National Committee, Parameswaran Iyer, Drinking Water & Sanitation, Ministry of Jal Shakti, State, central allocation &, State, State, JJM, CM, State, State, JJM, CM, the QS World University Rankings 2021, The Reserve Bank of India, Bank, JJM, Meghalaya, Assam & Bihar, Jharkhand &]


/usr/local/lib/python3.7/dist-packages/sklearn/svm/_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


OKKKKK
Possible answers list is  ['million', 'million', 'million', 'million', 'million']
final answer is million
Actual answer is USD 15 million
cosine similarity is  0.5773502691896258
Sequence matcher score is  0.6666666666666666
hits are  14 26 30
hits are 14
actual answers list is, ['Turkish lira', 'Birr', 'peso', 'Metical', 'Mark', 'Balboa', 'Ruble', 'Euro', 'Krona', 'Lake', 'Colon', 'Euro', 'Sudanese pound', 'Kina', 'koruna', 'Turkish lira', 'Deutsche Mark', 'Malaysian Dollar', 'Kuwaiti Dinar', 'Inti Soul', 'Drachma', 'pound', 'Bolivar', 'Rial', 'New Shekel', 'peso', 'Colon', 'pound', 'Tugrick', 'French Frank', 'American Dollar', 'Shilling', 'Rs. 1000', 'American Dollar', 'Rs 2 crore', 'Rs 540 crore', 'Rs. 115', 'Rs.5,000', '750-million-dollar', '50 pence', 'Rs.650 crore', '1 billion', 'USD 75 million', 'Rs.50,000 crores', 'Rs 2 trillion', 'Rs 2 trillion', 'Euro', '1,01,500', '1,8292 crores', 'USD 15 million']
answers that I got are ['Turkish', 'Birr', 'peso', 'metical', 'euro', 

In [ ]:
pip freeze


absl-py==0.12.0
alabaster==0.7.12
albumentations==0.1.12
altair==4.1.0
appdirs==1.4.4
argon2-cffi==20.1.0
arviz==0.11.2
astor==0.8.1
astropy==4.2.1
astunparse==1.6.3
async-generator==1.10
atari-py==0.2.9
atomicwrites==1.4.0
attrs==21.2.0
audioread==2.1.9
autograd==1.3
Babel==2.9.1
backcall==0.2.0
beautifulsoup4==4.6.3
bleach==3.3.0
blis==0.4.1
bokeh==2.3.2
Bottleneck==1.3.2
branca==0.4.2
bs4==0.0.1
CacheControl==0.12.6
cached-property==1.5.2
cachetools==4.2.2
catalogue==1.0.0
certifi==2021.5.30
cffi==1.14.5
cftime==1.5.0
chardet==3.0.4
click==7.1.2
cloudpickle==1.3.0
cmake==3.12.0
cmdstanpy==0.9.5
colorcet==2.0.6
colorlover==0.3.0
community==1.0.0b1
contextlib2==0.5.5
convertdate==2.3.2
coverage==3.7.1
coveralls==0.5
crcmod==1.7
cufflinks==0.17.3
cvxopt==1.2.6
cvxpy==1.0.31
cycler==0.10.0
cymem==2.0.5
Cython==0.29.23
daft==0.0.4
dask==2.12.0
datascience==0.10.6
debugpy==1.0.0
decorator==4.4.2
defusedxml==0.7.1
descartes==1.1.0
dill==0.3.4
distributed==1.25.3
dlib==19.18.0
dm-tree==0.1.